Встановлення бібліотеки

In [ ]:
!pip install scikit-surprise


  Using cached scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl


Імпорт бібліотек

In [ ]:
import numpy as np
import pandas as pd

from surprise import Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
from surprise import SVD, SVDpp, NMF

Завантаження датасету

In [ ]:
data = Dataset.load_builtin("ml-100k")


Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


Підбір гіперпараметрів для SVD

In [ ]:
param_grid = {
    "n_factors": [50, 100],
    "n_epochs": [20, 30],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.02, 0.05],
}

gs_svd = GridSearchCV(
    SVD,
    param_grid,
    measures=["rmse", "mae"],
    cv=3,
    n_jobs=-1
)

gs_svd.fit(data)

print("Best RMSE:", gs_svd.best_score["rmse"])
print("Best params (RMSE):", gs_svd.best_params["rmse"])
print("Best MAE:", gs_svd.best_score["mae"])
print("Best params (MAE):", gs_svd.best_params["mae"])


Best RMSE: 0.9318163758827623
Best params (RMSE): {'n_factors': 50, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.05}
Best MAE: 0.7356888848728728
Best params (MAE): {'n_factors': 50, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.05}


Підбір гіперпараметрів для SVD++

In [ ]:
param_grid_svdpp = {
    "n_factors": [50, 100],
    "n_epochs": [20],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.02, 0.05],
}

gs_svdpp = GridSearchCV(
    SVDpp,
    param_grid_svdpp,
    measures=["rmse", "mae"],
    cv=3,
    n_jobs=-1
)

gs_svdpp.fit(data)

print("SVD++ Best RMSE:", gs_svdpp.best_score["rmse"])
print("SVD++ Best params:", gs_svdpp.best_params["rmse"])


SVD++ Best RMSE: 0.9277516813929503
SVD++ Best params: {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


Підбір гіперпараметрів для NMF

In [ ]:
param_grid_nmf = {
    "n_factors": [30, 50],
    "n_epochs": [50],
    "reg_pu": [0.02, 0.06],
    "reg_qi": [0.02, 0.06],
}

gs_nmf = GridSearchCV(
    NMF,
    param_grid_nmf,
    measures=["rmse", "mae"],
    cv=3,
    n_jobs=-1
)

gs_nmf.fit(data)

print("NMF Best RMSE:", gs_nmf.best_score["rmse"])
print("NMF Best params:", gs_nmf.best_params["rmse"])

NMF Best RMSE: 0.9966624439954647
NMF Best params: {'n_factors': 30, 'n_epochs': 50, 'reg_pu': 0.06, 'reg_qi': 0.06}


Порівняння моделей

In [ ]:
results = pd.DataFrame([
    {"model": "SVD",   "rmse": gs_svd.best_score["rmse"],   "mae": gs_svd.best_score["mae"]},
    {"model": "SVD++", "rmse": gs_svdpp.best_score["rmse"], "mae": gs_svdpp.best_score["mae"]},
    {"model": "NMF",   "rmse": gs_nmf.best_score["rmse"],   "mae": gs_nmf.best_score["mae"]},
]).sort_values("rmse")

results


,model,rmse,mae
1,SVD++,0.927752,0.730163
0,SVD,0.931816,0.735689
2,NMF,0.996662,0.766659


Фінальна перевірка на тестовій вибірці

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

best_algo = SVDpp(**gs_svdpp.best_params["rmse"])

best_algo.fit(trainset)
preds = best_algo.test(testset)

print("Test RMSE:", accuracy.rmse(preds, verbose=False))
print("Test MAE:", accuracy.mae(preds, verbose=False))


Test RMSE: 0.9234319666330538
Test MAE: 0.7275974638685949


Параметри моделей підбирались за допомогою GridSearchCV з крос-валідацією. У результаті найкращу якість прогнозування продемонструвала модель SVD++, яка має найменші значення RMSE та MAE.

**ЗАВДАННЯ З ЗІРОЧКОЙ**

Імпорт бібліотек

In [ ]:
import numpy as np
from scipy.io import loadmat

Завантаження датасету

In [ ]:
data = loadmat("movies.mat")
Y = data["Y"]
R = data["R"]

num_movies, num_users = Y.shape

Ініціалізація параметрів моделі

In [ ]:
k = 20
alpha = 0.001
lambda_ = 0.1
epochs = 30

X = np.random.normal(0, 0.1, (num_movies, k))
W = np.random.normal(0, 0.1, (num_users, k))

Нормалізація рейтингів

In [ ]:
mean_rating = np.sum(Y) / np.sum(R)
Y_norm = (Y - mean_rating) * R

Функція втрат

In [ ]:
def compute_cost(X, W, Y, R, lambda_):
    pred = X @ W.T
    error = (pred - Y) * R
    cost = 0.5 * np.sum(error**2)
    cost += (lambda_ / 2) * (np.sum(X**2) + np.sum(W**2))
    return cost

Градієнти

In [ ]:
def compute_gradients(X, W, Y, R, lambda_):
    pred = X @ W.T
    error = (pred - Y) * R

    X_grad = error @ W + lambda_ * X
    W_grad = error.T @ X + lambda_ * W

    return X_grad, W_grad

Навчання градієнтним спуском

In [ ]:
for epoch in range(epochs):
    X_grad, W_grad = compute_gradients(X, W, Y, R, lambda_)
    X -= alpha * X_grad
    W -= alpha * W_grad

    if (epoch+1) % 10 == 0:
        print(epoch+1, compute_cost(X, W, Y, R, lambda_))

10 338534.48166267277
20 64672.52099824271
30 48175.124142521796


Завантаження назв фільмів

In [ ]:
def load_movie_names():
    with open("movie_ids.txt", encoding="ISO-8859-1") as f:
        return [line.strip().split(" ", 1)[1] for line in f]

movie_names = load_movie_names()

Рекомендації

In [ ]:
def recommend_movies(user_id, n=5):
    user_idx = user_id - 1
    scores = X @ W[user_idx]
    rated = R[:, user_idx]

    scores[rated == 1] = -np.inf

    top_idx = np.argsort(scores)[-n:][::-1]
    return [(movie_names[i], f"{scores[i]:.2f}") for i in top_idx]

In [ ]:
recommend_movies(user_id=1)

[('Close Shave, A (1995)', '4.91'),
 ("Schindler's List (1993)", '4.73'),
 ("One Flew Over the Cuckoo's Nest (1975)", '4.64'),
 ('Boot, Das (1981)', '4.63'),
 ('Third Man, The (1949)', '4.60')]